In [11]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy
import re
import pandas as pd

Load Data

In [12]:
bbc_data = pd.read_csv('bbc_news.csv')
bbc_data.head()

,Unnamed: 0,index,title,pubDate,guid,link,description
0,0,6684,Can I refuse to work?,"Wed, 10 Aug 2022 15:46:18 GMT",https://www.bbc.co.uk/news/business-62147992,https://www.bbc.co.uk/news/business-62147992?a...,With much of the UK enduring another period of...
1,1,9267,'Liz Truss the Brief?' World reacts to UK poli...,"Mon, 17 Oct 2022 11:35:12 GMT",https://www.bbc.co.uk/news/world-63285480,https://www.bbc.co.uk/news/world-63285480?at_m...,The UK's political chaos has been watched arou...
2,2,7387,Rationing energy is nothing new for off-grid c...,"Wed, 31 Aug 2022 05:20:18 GMT",https://www.bbc.co.uk/news/uk-scotland-highlan...,https://www.bbc.co.uk/news/uk-scotland-highlan...,Scoraig in the north west Highlands has long h...
3,3,767,The hunt for superyachts of sanctioned Russian...,"Tue, 22 Mar 2022 14:37:01 GMT",https://www.bbc.co.uk/news/60739336,https://www.bbc.co.uk/news/60739336?at_medium=...,"Wealthy Russians sanctioned by the US, EU and ..."
4,4,3712,Platinum Jubilee: 70 years of the Queen in 70 ...,"Wed, 01 Jun 2022 23:17:33 GMT",https://www.bbc.co.uk/news/uk-61660128,https://www.bbc.co.uk/news/uk-61660128?at_medi...,A quick look back at the Queen's 70 years on t...


In [13]:
bbc_data.describe()

,Unnamed: 0,index
count,1000.000000,1000.000000
mean,499.500000,7591.995000
std,288.819436,4576.727713
min,0.000000,4.000000
25%,249.750000,3619.250000
50%,499.500000,7318.000000
75%,749.250000,11679.250000
max,999.000000,15475.000000


In [14]:
bbc_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1000 non-null   int64 
 1   index        1000 non-null   int64 
 2   title        1000 non-null   object
 3   pubDate      1000 non-null   object
 4   guid         1000 non-null   object
 5   link         1000 non-null   object
 6   description  1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [15]:
titles = pd.DataFrame(bbc_data['title'])
titles.head()

,title
0,Can I refuse to work?
1,'Liz Truss the Brief?' World reacts to UK poli...
2,Rationing energy is nothing new for off-grid c...
3,The hunt for superyachts of sanctioned Russian...
4,Platinum Jubilee: 70 years of the Queen in 70 ...


Clean Data

In [16]:
# lowercase
titles['lowercase'] = titles['title'].str.lower()
titles.head()

,title,lowercase
0,Can I refuse to work?,can i refuse to work?
1,'Liz Truss the Brief?' World reacts to UK poli...,'liz truss the brief?' world reacts to uk poli...
2,Rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...
3,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...
4,Platinum Jubilee: 70 years of the Queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...


In [18]:
# remove stop words
en_stop = stopwords.words('english')
en_stop.remove('not')
titles['no_stopwords'] = titles['lowercase'].apply(lambda x: " ".join([word for word in x.split() if word not in en_stop])
)
titles.head()

,title,lowercase,no_stopwords
0,Can I refuse to work?,can i refuse to work?,refuse work?
1,'Liz Truss the Brief?' World reacts to UK poli...,'liz truss the brief?' world reacts to uk poli...,'liz truss brief?' world reacts uk political t...
2,Rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...,rationing energy nothing new off-grid community
3,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs
4,Platinum Jubilee: 70 years of the Queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years queen 70 seconds


In [20]:
# remove punctuation
titles['removed_punctuation'] = titles.apply(lambda x: re.sub(r'[^\w\s]', '', x['no_stopwords']), axis=1)
titles.head()


,title,lowercase,no_stopwords,removed_punctuation
0,Can I refuse to work?,can i refuse to work?,refuse work?,refuse work
1,'Liz Truss the Brief?' World reacts to UK poli...,'liz truss the brief?' world reacts to uk poli...,'liz truss brief?' world reacts uk political t...,liz truss brief world reacts uk political turmoil
2,Rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...,rationing energy nothing new off-grid community,rationing energy nothing new offgrid community
3,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs,hunt superyachts sanctioned russian oligarchs
4,Platinum Jubilee: 70 years of the Queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years queen 70 seconds,platinum jubilee 70 years queen 70 seconds


In [21]:
# tokenize
titles['tokens_clean'] = titles.apply(lambda x: word_tokenize(x['removed_punctuation']), axis=1)
titles.head()

,title,lowercase,no_stopwords,removed_punctuation,tokens_clean
0,Can I refuse to work?,can i refuse to work?,refuse work?,refuse work,"[refuse, work]"
1,'Liz Truss the Brief?' World reacts to UK poli...,'liz truss the brief?' world reacts to uk poli...,'liz truss brief?' world reacts uk political t...,liz truss brief world reacts uk political turmoil,"[liz, truss, brief, world, reacts, uk, politic..."
2,Rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...,rationing energy nothing new off-grid community,rationing energy nothing new offgrid community,"[rationing, energy, nothing, new, offgrid, com..."
3,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs,hunt superyachts sanctioned russian oligarchs,"[hunt, superyachts, sanctioned, russian, oliga..."
4,Platinum Jubilee: 70 years of the Queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years queen 70 seconds,platinum jubilee 70 years queen 70 seconds,"[platinum, jubilee, 70, years, queen, 70, seco..."


In [23]:
# lemmatizing
lm = WordNetLemmatizer()
titles['lemmatized'] = titles['tokens_clean'].apply(lambda tokens: [lm.lemmatize(token) for token in tokens])
titles.head()

,title,lowercase,no_stopwords,removed_punctuation,tokens_clean,lemmatized
0,Can I refuse to work?,can i refuse to work?,refuse work?,refuse work,"[refuse, work]","[refuse, work]"
1,'Liz Truss the Brief?' World reacts to UK poli...,'liz truss the brief?' world reacts to uk poli...,'liz truss brief?' world reacts uk political t...,liz truss brief world reacts uk political turmoil,"[liz, truss, brief, world, reacts, uk, politic...","[liz, truss, brief, world, reacts, uk, politic..."
2,Rationing energy is nothing new for off-grid c...,rationing energy is nothing new for off-grid c...,rationing energy nothing new off-grid community,rationing energy nothing new offgrid community,"[rationing, energy, nothing, new, offgrid, com...","[rationing, energy, nothing, new, offgrid, com..."
3,The hunt for superyachts of sanctioned Russian...,the hunt for superyachts of sanctioned russian...,hunt superyachts sanctioned russian oligarchs,hunt superyachts sanctioned russian oligarchs,"[hunt, superyachts, sanctioned, russian, oliga...","[hunt, superyachts, sanctioned, russian, oliga..."
4,Platinum Jubilee: 70 years of the Queen in 70 ...,platinum jubilee: 70 years of the queen in 70 ...,platinum jubilee: 70 years queen 70 seconds,platinum jubilee 70 years queen 70 seconds,"[platinum, jubilee, 70, years, queen, 70, seco...","[platinum, jubilee, 70, year, queen, 70, second]"


In [24]:
# create tokens list
token_clean_list = sum(titles['lemmatized'], [])
token_clean_list

['refuse',
 'work',
 'liz',
 'truss',
 'brief',
 'world',
 'reacts',
 'uk',
 'political',
 'turmoil',
 'rationing',
 'energy',
 'nothing',
 'new',
 'offgrid',
 'community',
 'hunt',
 'superyachts',
 'sanctioned',
 'russian',
 'oligarch',
 'platinum',
 'jubilee',
 '70',
 'year',
 'queen',
 '70',
 'second',
 'red',
 'bull',
 'found',
 'guilty',
 'breaking',
 'formula',
 '1',
 'budget',
 'cap',
 'world',
 'triathlon',
 'championship',
 'series',
 'flora',
 'duffy',
 'beat',
 'georgia',
 'taylorbrown',
 'womens',
 'title',
 'terry',
 'hall',
 'coventry',
 'scooter',
 'rideout',
 'pay',
 'tribute',
 'singer',
 'post',
 'office',
 'fujitsu',
 'face',
 'inquiry',
 'horizon',
 'scandal',
 'pavement',
 'parking',
 'frightens',
 'me',
 'uk',
 'interest',
 'rate',
 'rise',
 'affect',
 'high',
 'could',
 'go',
 'stayed',
 'storm',
 'happens',
 'now',
 'six',
 'nation',
 'scotland',
 'best',
 'since',
 '99',
 'beat',
 'best',
 'ireland',
 'ever',
 'long',
 'liz',
 'truss',
 'survive',
 'prime',
 'm

POS tagging

In [27]:
nlp = spacy.load('en_core_web_sm')

In [28]:
spacy_doc = nlp(' '.join(token_clean_list))

In [26]:
pos_df = pd.DataFrame(columns=['token', 'pos_tag'])

In [37]:
for token in spacy_doc:
    pos_df = pd.concat([pos_df, pd.DataFrame.from_records(
    [{'token': token.text, 'pos_tag': token.pos_}]
    )], ignore_index=True)

In [30]:
#token frequency
pos_df_counts = pos_df.groupby(['token', 'pos_tag']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)
pos_df_counts.head(10)

,token,pos_tag,counts
30,2022,NUM,47
1162,england,PROPN,45
870,cup,PROPN,39
3707,uk,PROPN,37
3055,say,VERB,37
3840,war,NOUN,34
2386,new,ADJ,31
3948,world,NOUN,30
3949,world,PROPN,26
3710,ukraine,PROPN,23


In [32]:
verbs = pos_df_counts[pos_df_counts.pos_tag == 'VERB'][0:10]
verbs

,token,pos_tag,counts
3055,say,VERB,37
3711,ukraine,VERB,22
1651,hit,VERB,13
2133,make,VERB,13
1459,get,VERB,13
358,beat,VERB,13
1380,found,VERB,13
3460,take,VERB,13
1473,give,VERB,11
3918,win,VERB,10


Named Entity Recognition

In [33]:
ner_df = pd.DataFrame(columns=['token', 'ner_tag'])

In [35]:
for token in spacy_doc.ents:
    if pd.isna(token.label_) is False:
        ner_df = pd.concat([ner_df, pd.DataFrame.from_records(
            [{'token': token.text, 'ner_tag': token.label_}]
        )])

In [39]:
ner_df.head()

,token,ner_tag
0,russian,NORP
0,70 year,DATE
0,70 second,TIME
0,bull,ORG
0,1,CARDINAL


In [41]:
# token frequency count
ner_df_counts = ner_df.groupby(['token', 'ner_tag']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)
ner_df_counts.head(10)

,token,ner_tag,counts
34,2022,CARDINAL,30
451,russian,NORP,25
223,first,ORDINAL,15
35,2022,DATE,11
233,france,GPE,10
450,russia,GPE,10
519,tory,NORP,9
339,london,GPE,9
4,10,CARDINAL,8
409,one,CARDINAL,8


In [43]:
# most common people
people = ner_df_counts[ner_df_counts.ner_tag == 'PERSON'][0:10]
people

,token,ner_tag,counts
432,putin,PERSON,7
127,boris johnson,PERSON,5
106,antonio conte,PERSON,3
102,andy murray,PERSON,3
524,tyre nichols,PERSON,2
581,zelensky,PERSON,2
249,harry,PERSON,2
254,hodgkinson,PERSON,2
252,harry meghan,PERSON,2
331,lenny henry,PERSON,2


In [48]:
# most common places

places = ner_df_counts[ner_df_counts.ner_tag == 'GPE'][0:10]
places

,token,ner_tag,counts
233,france,GPE,10
450,russia,GPE,10
339,london,GPE,9
531,uk,GPE,8
335,liverpool,GPE,7
211,england,GPE,7
156,china,GPE,7
109,australia,GPE,7
479,south africa,GPE,7
261,india,GPE,6
